## 对比 ( 直接drop非数值列  、 one-hot处理非数值列 ） 

1.drop 和 one-hot

2.align training set & test set

3.missing data 处理

4.cross validation

### data input

In [10]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [11]:
training_predictors = train_data.loc[:, train_data.columns != 'SalePrice']
target = train_data.SalePrice

test_predictors = test_data

### drop 和 one-hot


In [12]:
# Drop those non_numeric columns
numeric_training_predictors = training_predictors.select_dtypes(exclude=["object"]).drop("Id",1)
numeric_test_predictors = test_predictors.select_dtypes(exclude=["object"]).drop("Id",1)

# One-hot
one_hot_training_predictors = pd.get_dummies(training_predictors).drop("Id",1)
one_hot_test_predictors = pd.get_dummies(test_predictors).drop("Id",1)

### align training set & test set

In [13]:
one_hot_training_predictors, one_hot_test_predictors = one_hot_training_predictors.align(one_hot_test_predictors, join='left', axis=1)

### missing data 处理

In [14]:
# numeric_predictors
from sklearn.preprocessing import Imputer
my_imputer = Imputer()

cols_with_missing = (col for col in numeric_training_predictors.columns if numeric_training_predictors[col].isnull().any())
for col in cols_with_missing:
    numeric_training_predictors[col+'_was_missing'] = numeric_training_predictors[col].isnull()
numeric_training_predictors = my_imputer.fit_transform(numeric_training_predictors)

cols_with_missing = (col for col in numeric_test_predictors.columns if numeric_test_predictors[col].isnull().any())
for col in cols_with_missing:
    numeric_test_predictors[col+'_was_missing'] = numeric_test_predictors[col].isnull()
numeric_test_predictors = my_imputer.fit_transform(numeric_test_predictors)

In [15]:
# one_hot_predictors
cols_with_missing = (col for col in one_hot_training_predictors.columns if one_hot_training_predictors[col].isnull().any())
for col in cols_with_missing:
    one_hot_training_predictors[col+'_was_missing'] = one_hot_training_predictors[col].isnull()
one_hot_training_predictors = my_imputer.fit_transform(one_hot_training_predictors)

cols_with_missing = (col for col in one_hot_test_predictors.columns if one_hot_test_predictors[col].isnull().any())
for col in cols_with_missing:
    one_hot_test_predictors[col+'_was_missing'] = one_hot_test_predictors[col].isnull()
one_hot_test_predictors = my_imputer.fit_transform(one_hot_test_predictors)

### Cross Validation

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

In [17]:
get_mae(numeric_training_predictors,target)

18535.641690425688

In [18]:
get_mae(one_hot_training_predictors,target)

18105.271256369306